In [122]:
import pandas as pd
import numpy as np
from datetime import datetime
df = pd.read_excel("C:\\Users\\Admin\\Documents\\MATLAB\\eeglab2023.1\\EyeTrackerData.xlsx")

In [123]:
df = df.loc[:,["Sensor Time", "Lft X Pos", "Lft Y Pos", "Lft Pupil Diameter","Rt X Pos", "Rt Y Pos","Rt Pupil Diameter","Capture Time"]]

In [124]:
df['gaze_x'] = (df['Lft X Pos'] + df["Rt X Pos"])/2
df['gaze_y'] = (df['Lft Y Pos'] + df["Rt Y Pos"])/2

In [125]:
df['time_convert'] = df['Capture Time'].apply(lambda x: datetime.utcfromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S'))

In [126]:
df['time_convert'] = df['time_convert'].apply(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))

In [127]:
def blink_detection(x, y, time, missing=0.0, minlen=10):

    """Detects blinks, defined as a period of missing data that lasts for at
    least a minimal amount of samples

    arguments
    x		-	numpy array of x positions
    y		-	numpy array of y positions
    time		-	numpy array of EyeTribe timestamps
    keyword arguments
    missing	-	value to be used for missing data (default = 0.0)
    minlen	-	integer indicating the minimal amount of consecutive
                missing samples

    returns
    Sblk, Eblk, total_duration
                Sblk	-	list of lists, each containing [starttime]
                Eblk	-	list of lists, each containing [starttime, endtime, duration]
                total_duration - total duration of blinks
    """

    # empty list to contain data
    Sblk = []
    Eblk = []
    total_duration = 0

    # check where the missing samples are
    mx = np.array(x==missing, dtype=int)
    my = np.array(y==missing, dtype=int)
    miss = np.array((mx+my) == 2, dtype=int)

    # check where the starts and ends are (+1 to counteract shift to left)
    diff = np.diff(miss)
    starts = np.where(diff==1)[0] + 1
    ends = np.where(diff==-1)[0] + 1

    # compile blink starts and ends
    for i in range(len(starts)):
        # get starting index
        s = starts[i]
        # get ending index
        if i < len(ends):
            e = ends[1+i]
        # elif len(ends) > 0:
        #     e = ends[-1]
        else:
            e = -1
        # append only if the duration in samples is equal to or greater than
        # the minimal duration
        if e-s>= minlen:
            # add starting time
            Sblk.append([time[s]])
            # add ending time and calculate duration
            duration = (time[e] - time[s]).seconds/1000
            total_duration += duration
            Eblk.append([time[s], time[e], total_duration])

    return Sblk, Eblk, total_duration

# Call the blink_detection function
Sblk, Eblk, total_duration = blink_detection(df['gaze_x'], df['gaze_y'], df['time_convert'], minlen=10)
num_blinks = len(Eblk) 
blink_rate = len(Eblk) /((df['time_convert'].iloc[-1] - df['time_convert'].iloc[0]).total_seconds()/60000)
n=total_duration/num_blinks
# Output results
print("Sblk:", Sblk)
print("Eblk:", Eblk)
print("Total number of blinks in the given data :", num_blinks)
print("Average blink duration :", n)
print("Blink rate (blinks per Minute):", blink_rate)

Sblk: [[Timestamp('1972-04-10 22:20:54')], [Timestamp('1972-04-10 23:45:16')], [Timestamp('1972-04-11 00:11:25')], [Timestamp('1972-04-11 02:19:53')], [Timestamp('1972-04-11 03:10:12')], [Timestamp('1972-04-11 05:33:42')], [Timestamp('1972-04-11 08:09:46')], [Timestamp('1972-04-11 09:28:32')], [Timestamp('1972-04-11 11:45:32')], [Timestamp('1972-04-11 12:52:04')], [Timestamp('1972-04-11 13:05:15')], [Timestamp('1972-04-11 13:08:48')], [Timestamp('1972-04-11 13:17:45')], [Timestamp('1972-04-11 13:22:57')], [Timestamp('1972-04-11 13:29:58')], [Timestamp('1972-04-11 14:35:25')], [Timestamp('1972-04-11 23:01:06')], [Timestamp('1972-04-12 01:32:58')], [Timestamp('1972-04-12 01:49:47')], [Timestamp('1972-04-12 05:19:05')], [Timestamp('1972-04-12 06:28:18')], [Timestamp('1972-04-12 07:19:12')], [Timestamp('1972-04-12 07:49:29')], [Timestamp('1972-04-12 08:20:47')], [Timestamp('1972-04-12 09:54:51')], [Timestamp('1972-04-12 10:08:29')], [Timestamp('1972-04-12 10:21:23')], [Timestamp('1972-04-1

In [128]:
def remove_missing(x, y, time, missing):
	mx = np.array(x==missing, dtype=int)
	my = np.array(y==missing, dtype=int)
	x = x[(mx+my) != 2]
	y = y[(mx+my) != 2]
	time = time[(mx+my) != 2]
	return x, y, time

In [129]:
def fixation_detection(x, y, time, missing=0.0, maxdist=25, mindur=100):
    x, y, time = remove_missing(x, y, time, missing)

    Sfix = []
    Efix = []

    si = 0
    fixstart = False
    for i in range(1, len(x)):
        try:
            squared_distance = ((x.iloc[si] - x.iloc[i]) ** 2 + (y.iloc[si] - y.iloc[i]) ** 2)
            dist = 0.0
            if squared_distance > 0:
                dist = squared_distance ** 0.5
            if dist <= maxdist and not fixstart:
                si = 0 + i
                fixstart = True
                Sfix.append([time.iloc[i]])  # Adjusted indexing here
            elif dist > maxdist and fixstart:
                fixstart = False
                if int((time.iloc[i - 1] - Sfix[-1][0]).total_seconds() * 1000) >= mindur:
                    Efix.append([Sfix[-1][0], time.iloc[i - 1], int((time.iloc[i - 1] - Sfix[-1][0]).total_seconds() * 1000), x.iloc[si], y.iloc[si]])  # Adjusted indexing here
                else:
                    Sfix.pop(-1)
                si = 0 + i
            elif not fixstart:
                si += 1
        except IndexError:
            pass

    return Efix
# Load the data
df = pd.read_excel('C:\\Users\\Admin\\Documents\\MATLAB\\eeglab2023.1\\EyeTrackerData.xlsx')
df = df.loc[:,["Sensor Time", "Lft X Pos", "Lft Y Pos", "Lft Pupil Diameter","Rt X Pos", "Rt Y Pos","Rt Pupil Diameter","Capture Time"]]
df['gaze_x'] = (df['Lft X Pos'] + df["Rt X Pos"])/2
df['gaze_y'] = (df['Lft Y Pos'] + df["Rt Y Pos"])/2
df['time_convert'] = df['Capture Time'].apply(lambda x: datetime.utcfromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S'))
df['time_convert'] = df['time_convert'].apply(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))

In [130]:
fixation_detection(df['gaze_x'], df['gaze_y'], df['time_convert'], missing=0.0, maxdist=25, mindur=100)

[[Timestamp('1972-04-10 22:17:18'),
  Timestamp('1972-04-10 22:20:33'),
  195000,
  121.9692,
  86.73845],
 [Timestamp('1972-04-10 22:24:37'),
  Timestamp('1972-04-10 22:24:55'),
  18000,
  331.1363,
  295.98635],
 [Timestamp('1972-04-10 22:25:31'),
  Timestamp('1972-04-10 22:34:27'),
  536000,
  638.4146,
  596.6715],
 [Timestamp('1972-04-10 22:34:54'),
  Timestamp('1972-04-10 23:03:50'),
  1736000,
  620.2367,
  613.7307],
 [Timestamp('1972-04-10 23:07:24'),
  Timestamp('1972-04-10 23:08:41'),
  77000,
  632.2733,
  611.7738],
 [Timestamp('1972-04-10 23:09:22'),
  Timestamp('1972-04-10 23:11:14'),
  112000,
  662.406,
  608.464],
 [Timestamp('1972-04-10 23:11:53'),
  Timestamp('1972-04-10 23:14:30'),
  157000,
  686.6879,
  596.0922],
 [Timestamp('1972-04-10 23:14:54'),
  Timestamp('1972-04-10 23:16:59'),
  125000,
  654.0575,
  600.1948],
 [Timestamp('1972-04-10 23:17:20'),
  Timestamp('1972-04-10 23:18:58'),
  98000,
  624.6464,
  601.4726],
 [Timestamp('1972-04-10 23:19:34'),
  Ti

In [131]:

fixations = fixation_detection(df['gaze_x'], df['gaze_y'], df['time_convert'])

total_fixation_duration = sum((end - start).total_seconds() for start, end, duration, _, _ in fixations)

average_fixation_duration = total_fixation_duration / len(fixations)

# Total fixation count
total_fixation_count = len(fixations)

print("Average Fixation Duration (seconds):", average_fixation_duration)
print("Total Fixation Count:", total_fixation_count)

Average Fixation Duration (seconds): 156.06755126658624
Total Fixation Count: 829
